In [ ]:
import tweepy
import json
import requests
import time
from datetime import date
from datetime import datetime 

#Authenticate to Twitter
API_KEY = api_key
API_SECRET_KEY = api_secret_key
BEARER_TOKEN = bearer_token
ACCESS_TOKEN = access_token
ACCESS_TOKEN_SECRET = access_token_secret

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#Keeps track of already replied-to tweets
old_twts = []
today = date.today()
date = today.strftime("%d-%m-%Y")


def vaccine_info(pincode):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = 'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode=' + pincode + '&date=' + date

    response = requests.get(url, headers=headers)

    data = response.json()
    content = response.text
    
    if 'Invalid Pincode' in content:
        msg = 'Pincode is invalid. Try again!'
    else:
        centers = data['centers']
        count = 0
        if len(centers) > 0:
            data_list = []
            for cen in centers:
                sessions = cen['sessions']
                for each in sessions:
                    if each['available_capacity'] > 0 and count < 1:
                        data_list.append({'Center Name': cen['name'], 'Center Address': cen['address'], 'Availability': each['available_capacity'], 'Date': each['date']})
                        count += 1
                        parse_data = json.dumps(data_list[0])
                        parse_data = parse_data.replace('{', '')
                        parse_data = parse_data.replace('}', '\n')
                        parse_data = parse_data.replace('[', '')
                        parse_data = parse_data.replace(']', '')
                        parse_data = parse_data.replace('"', '')
                        msg = parse_data
                    elif each['available_capacity'] == 0 and count < 1:
                        msg = 'Currently no appointments could be located. Please check back later! Stay safe!'
                    
        else:
            msg = 'Currently no centers could be located. Please check back later! Stay safe!'
    
    return msg

def check_mentions(api, keywords, since_id):
    new_since_id = since_id

    for tweet in tweepy.Cursor(api.mentions_timeline, since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)

        #Checking if tweet is an original or reply
        if tweet.in_reply_to_status_id is not None:
            continue
        
        #Sending reply to mentioned tweets
        if any(keyword in tweet.text.lower() for keyword in keywords):
            if tweet.id not in old_twts:
                old_twts.append(tweet.id)
                
                content = tweet.text.split(" ")
                if len(content) > 0 and len(content) < 5:
                    pincode = content[3]
                else:
                    msg = "Incorrect formatting led to processing errors!"

                msg = vaccine_info(pincode)

                #Reply tweet contents
                api.update_status(
                    status = msg,
                    in_reply_to_status_id = tweet.id, auto_populate_reply_metadata=True
                )
                print('Replied to tweet!')
            else:
                pass

    return new_since_id


def main():
    #Create API Object
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    since_id = 1

    run = True
    while run:
        since_id = check_mentions(api, ["vaccines in"], since_id)
        print(old_twts)
        time.sleep(45)
    
main()